In [5]:
import cv2, os, subprocess, random
import pandas as pd
import numpy as np, matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor, as_completed
from matplotlib import image as mpimg
from glob import glob

### Path to the directory containing Open FACS code

In [6]:
# Path to the OpenFace executable for facial feature extraction
FACE_DETECTION_OPENFACS = "P:/Dissertation/Models/Face_Detection_Model/OpenFace_2.2.0_win_x64/FeatureExtraction.exe"

### Path to the directory containing DPD dataset images

In [7]:
# Path to the directory containing DPD dataset images
DPD_IMAGES = "C:/Users/psxah20/Desktop/Dissertation/Final_cropped_pictures/Cropped_Faces/Expression_Images"

# Path to the Excel file containing previously identified Action Units (AUs) for DPD
DPD_EXCEL = "C:/Users/psxah20/Desktop/Dissertation/DPD_OPENFACS/DelawarePainDatabase_StimulusCharacterization_forOSF.xlsx"

# Path to the directory where OpenFace output for DPD images will be saved
OPENFACE_IMG_DPD = 'C:/Users/psxah20/Desktop/Dissertation/DPD_OPENFACS/Landmarks_Detected/'

# Path to the CSV file where the OpenFace results for DPD images will be stored
DPD_OPENFACE_CSV = 'C:/Users/psxah20/Desktop/Dissertation/DPD_OPENFACS/DPD_OPENFACS.csv'

# Path to the CSV file where the OpenFace results for UNBC images will be stored
UNBC_OPENFACE_CSV = 'C:/Users/psxah20/Desktop/Dissertation/UNBC_OPENFACS/UNBC_OPENFACS.csv'

# List of all image file paths in the DPD dataset (including .jpg files) using glob
DPD_IMAGES_PATH = glob(os.path.join(DPD_IMAGES , '**', '*.jpg'), recursive=True)

# Output the number of image paths found in the DPD dataset
len(DPD_IMAGES_PATH)

577

### Path to the directory containing UNBC dataset images

In [8]:
# Root path of the UNBC-McMaster dataset
ROOT_UNBC = "C:/Users/psxah20/Desktop/UNBC-McMaster/"

# Path to the directory containing UNBC images
UNBC_IMAGES = ROOT_UNBC + "Images/"

# List of all image file paths in the UNBC dataset, including both .jpg and .png files
UNBC_IMGS_PATH = [values.replace('\\','/') for values in 
                 (glob(os.path.join(UNBC_IMAGES , '**', '*.jpg'), recursive=True) + 
                  glob(os.path.join(UNBC_IMAGES , '**', '*.png'), recursive=True))]

# Path to the directory containing FACS coded information for UNBC images
UNBC_FACS = ROOT_UNBC + "Frame_Labels/FACS/"

# List of all FACS-coded text file paths in the UNBC dataset
FACS_TEXT = [values.replace('\\','/') for values in glob(os.path.join(UNBC_FACS , '**', '*.txt'), recursive=True)]

# Path to the directory containing PSPI coded information for UNBC images
UNBC_PSPI = ROOT_UNBC + "Frame_Labels/PSPI/"

# List of all PSPI-coded text file paths in the UNBC dataset
PSPI_TEXT = [values.replace('\\','/') for values in glob(os.path.join(UNBC_PSPI , '**', '*.txt'), recursive=True)]

# Path to the directory where OpenFace output for UNBC images will be saved
OPENFACE_IMG_UNBC = 'C:/Users/psxah20/Desktop/Dissertation/UNBC_OPENFACS/Landmarks_Detected/'

len(UNBC_IMGS_PATH)

48398

### DPD previously identified AUs

In [9]:
# Load the Excel file containing previously identified Action Units (AUs) for DPD into a DataFrame.
# The data is loaded from the first sheet (sheet_name=0), and the 'Target' column is used as the index.
original_facs_dpd = pd.read_excel(io=DPD_EXCEL, 
                                  sheet_name=0,
                                  index_col='Target')

# Load the Excel file containing manually identified Action Units (AUs) for DPD into a DataFrame.
# The data is loaded from the fifth sheet (sheet_name=4), and the 'Target' column is used as the index.
manual_facs_dpd = pd.read_excel(io=DPD_EXCEL, 
                                sheet_name=4,
                                index_col='Target')

c:\Anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [10]:
# Display the first few rows of the DataFrame to verify the data has been loaded correctly
manual_facs_dpd.head()

,Expession_Highest,Pain_Expression,"AU(4,6,7,9,45)_PainIndex",AU01_m,AU02_m,AU04_m,AU05_m,AU06_m,AU07_m,AU09_m,...,AU12_m,AU14_m,AU15_m,AU17_m,AU20_m,AU23_m,AU25_m,AU26_m,AU28_m,AU45_m
Target,,,,,,,,,,,,,,,,,,,,,
DPD_1_AF12_p4c,surprised,2.285714,2,1,0,1,0,0,1,0,...,1,0,0,0,0,0,1,1,0,0
DPD_1_AF13_p2c,pain,3.292683,3,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
DPD_1_AF15_p5c,pain,4.463415,3,0,0,1,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
DPD_1_AF16_p3c,pain,3.904762,3,0,0,1,0,0,0,1,...,0,1,0,0,0,1,0,0,0,1
DPD_1_AF17_p2c,pain,4.000000,3,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1


In [11]:
# Display the first few rows of the DataFrame to verify the data has been loaded correctly
original_facs_dpd.head()

,Consent,Race,Gender,Masculine_Neutral,Feminine_Neutral,BabyFaced_Neutral,Trustworthy_Neutral,Dominant_Neutral,Attractive_Neutral,Unusual_Neutral,...,AU12_OF,AU14_OF,AU15_OF,AU17_OF,AU20_OF,AU23_OF,AU25_OF,AU26_OF,AU28_OF,AU45_OF
Target,,,,,,,,,,,,,,,,,,,,,
DPD_1_OF8_p2c,publication materials/website materials,Arabic,Female,3.268293,3.756098,2.463415,3.243902,2.731707,2.560976,3.317073,...,1,1,1,0,0,1,1,0,0,1
DPD_1_OF8_p3c,publication materials/website materials,Arabic,Female,3.268293,3.756098,2.463415,3.243902,2.731707,2.560976,3.317073,...,1,1,1,1,0,1,0,0,0,1
DPD_1_OF8_p4c,publication materials/website materials,Arabic,Female,3.268293,3.756098,2.463415,3.243902,2.731707,2.560976,3.317073,...,1,1,0,1,0,0,0,0,0,1
DPD_1_AF6_nosering_p2c,full,Asian,Female,2.224138,5.413793,4.051724,4.603448,3.206897,3.862069,3.258621,...,0,0,0,1,0,0,0,0,0,0
DPD_1_AF6_nosering_p4c,full,Asian,Female,2.224138,5.413793,4.051724,4.603448,3.206897,3.862069,3.258621,...,0,0,0,1,0,0,1,0,0,0


### UNBC previously identified AUs

In [12]:
def process_facs_file(facs):
    """
    Processes a single FACS file by reading its contents, corresponding PSPI file,
    and returns two DataFrames: one for the FACS data and one for the PSPI data.

    Parameters:
    - facs (str): Path to the FACS file.

    Returns:
    - df (DataFrame): Processed DataFrame for the FACS file.
    - original_facs_unbc_pspi (DataFrame): DataFrame containing the PSPI score.
    """
    # Extract the user ID from the filename
    user_id = facs.split('/')[-1].split('.')[0]

    # Open and read the content of the FACS file
    with open(facs, 'r', encoding='utf-8') as file:
        content = file.read()

    # Skip processing if the file is empty
    if content == "":
        return None, None

    # Read the FACS file into a DataFrame with specified column names
    df = pd.read_csv(facs, delim_whitespace=True, header=None, names=["AU_Number", "AU_Intensity", "AU_Onset", "AU_Offset"])

    # Initialize an empty DataFrame for PSPI data
    original_facs_unbc_pspi = pd.DataFrame()

    # Set a MultiIndex for the DataFrame using 'user_id' and 'AU_Number'
    df.index = pd.MultiIndex.from_product([[user_id], df["AU_Number"]], names=["user_id", "row_index"])

    # Read the corresponding PSPI file and add it as a new column "PSPI_Score"
    original_facs_unbc_pspi.loc[user_id, "PSPI_Score"] = pd.read_csv(facs.replace('FACS', 'PSPI'), delim_whitespace=True, header=None).iloc[0, 0]

    # Drop the "AU_Number" column as it is now part of the index
    df.drop("AU_Number", axis=1, inplace=True)

    return df, original_facs_unbc_pspi

def process_all_facs_files(FACS_TEXT, max_workers=8):
    """
    Processes multiple FACS files in parallel and concatenates the resulting DataFrames.

    Parameters:
    - FACS_TEXT (list): List of FACS file paths.
    - max_workers (int): Number of parallel workers.

    Returns:
    - original_facs_unbc (DataFrame): Combined DataFrame of all processed FACS files.
    - original_facs_unbc_pspi (DataFrame): Combined DataFrame of all processed PSPI scores.
    """
    # Initialize lists to hold DataFrames for FACS and PSPI data
    df_list = []
    pspi_list = []

    # Use ThreadPoolExecutor to process files in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_facs_file, facs) for facs in FACS_TEXT]

        # As the futures complete, append the resulting DataFrames to the lists
        for future in as_completed(futures):
            df, pspi = future.result()
            if df is not None:
                df_list.append(df)
            if pspi is not None:
                pspi_list.append(pspi)

    # Concatenate all individual DataFrames into one combined DataFrame
    original_facs_unbc = pd.concat(df_list, ignore_index=False)
    original_facs_unbc_pspi = pd.concat(pspi_list, ignore_index=False)

    return original_facs_unbc, original_facs_unbc_pspi

# Concatenate all individual DataFrames into one combined DataFrame
original_facs_unbc, original_facs_unbc_pspi = process_all_facs_files(FACS_TEXT, max_workers=1024)

# Display the first few rows of the combined DataFrame to verify the result
original_facs_unbc.head()

AU_Intensity  AU_Onset  AU_Offset
user_id              row_index                                   
jk103t1aiaff059_facs 12.0                3.0       0.0        0.0
                     26.0                4.0       0.0        0.0
vw121t1aeaff121_facs 12.0                1.0       0.0        0.0
jl047t1aiaff202_facs 12.0                3.0       0.0        0.0
vw121t1aeaff120_facs 12.0                1.0       0.0        0.0

### Let's save the UNBC xslx sheet

In [16]:
# Define the output Excel file path
output_file = "C:/Users/psxah20/Desktop/Dissertation/UNBC_OPENFACS/UNBC_Previous_Study.xlsx"

# Create a Pandas Excel writer using XlsxWriter as the engine
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    # Write each dataframe to a different worksheet, including the index
    original_facs_unbc.to_excel(writer, sheet_name='Sheet1', index=True)
    original_facs_unbc_pspi.to_excel(writer, sheet_name='Sheet2', index=True, index_label='Target')

### Function Descriptions:
- `process_image(face_image, landmarkds_op, FACE_DETECTION_OPENFACS)`: Processes a single image by creating necessary directories, setting up the output paths, and running face detection using the specified executable.
- `process_images_in_parallel(image_paths, replace_from, replace_to, FACE_DETECTION_OPENFACS, workers=64)`: Processes multiple images in parallel using `ThreadPoolExecutor` to expedite the face detection process across a list of images.

In [9]:
def process_image(face_image, landmarkds_op, FACE_DETECTION_OPENFACS):
    """
    Processes a single image by setting up the output directories and running face detection.
    
    Parameters:
    - face_image (str): Path to the input image file.
    - landmarkds_op (str): Path to save the output of landmark detection.
    - FACE_DETECTION_OPENFACS (str): Path to the face detection executable.
    """
    # Ensure the output directory for the landmarks exists
    os.makedirs(name='/'.join(landmarkds_op.split('/')[:-1]), exist_ok=True)

    # Construct and standardize the output directory path for landmarks
    output_dir_landmarks = landmarkds_op.split('.')[0]
    output_dir_landmarks = landmarkds_op.replace('\\', '/')
    print(output_dir_landmarks)
    os.makedirs(output_dir_landmarks, exist_ok=True)

    # Prepare the command to run the face detection tool with specified options
    command = [
        FACE_DETECTION_OPENFACS,
        '-f', face_image.replace("\\", "/"),
        '-out_dir', output_dir_landmarks.replace("\\", "/"),
        '-vis-track', output_dir_landmarks.replace("\\", "/"),
        '-vis-hog', output_dir_landmarks.replace("\\", "/"),
        '-vis-aus', output_dir_landmarks.replace("\\", "/"),
    ]

    # Execute the command and handle any errors
    try:
        subprocess.run(command, check=True, capture_output=True, text=True)
        print(f"Successfully processed {face_image}")
    except subprocess.CalledProcessError as e:
        print(f"Error processing {face_image}: {e}")
        print(e.stderr)
    except FileNotFoundError as e:
        print(f"Executable not found: {FACE_DETECTION_OPENFACS}")
        print(e)

def process_images_in_parallel(image_paths, replace_from, replace_to, FACE_DETECTION_OPENFACS, workers=64):
    """
    Processes multiple images in parallel using ThreadPoolExecutor.
    
    Parameters:
    - image_paths (list): List of image file paths to process.
    - replace_from (str): String to be replaced in the output path.
    - replace_to (str): String to replace with in the output path.
    - FACE_DETECTION_OPENFACS (str): Path to the face detection executable.
    - workers (int): Number of parallel workers (default is 64).
    """
    # Use ThreadPoolExecutor to process images concurrently
    with ThreadPoolExecutor(max_workers=workers) as executor:
        futures = []
        for face_image in image_paths:
            # Construct the output path by replacing part of the image path
            output_path = face_image.replace(replace_from, replace_to).replace('\\', '/')
            # Submit the image processing task to the executor
            futures.append(executor.submit(process_image, face_image, output_path, FACE_DETECTION_OPENFACS))
        
        # Wait for all submitted tasks to complete
        for future in as_completed(futures):
            future.result()  # Retrieve result to catch exceptions if any

In [10]:
# Let's process for DPD Dataset
process_images_in_parallel(
    DPD_IMAGES_PATH,  # List of all image file paths in the DPD dataset
    'Final_cropped_pictures/Cropped_Faces/Expression_Images',  # Part of the path to be replaced in the output paths
    'DPD_OPENFACS/Landmarks_Detected',  # Replacement path where the output will be saved
    FACE_DETECTION_OPENFACS,  # Path to the face detection executable
    workers=180  # Number of parallel workers to use for processing
)

##### The `merge_openface_csvs` function merges multiple CSV files from a specified directory into a single DataFrame and saves the result as a new CSV file. It also adds a column with the subject's name, which is derived from the original file name.

In [12]:
def merge_openface_csvs(image_path, csv_save_path, max_workers=1024):
    """
    Merges multiple CSV files from a specified directory into a single DataFrame 
    and saves the result to a new CSV file, utilizing parallel processing to speed up the process.
    
    Parameters:
    - image_path (str): Directory path where CSV files are located.
    - csv_save_path (str): Path where the merged CSV file will be saved.
    - max_workers (int): Maximum number of threads to use for parallel processing.
    """
    # Initialize an empty list to store DataFrames
    df_list = []

    # Generate a list of all CSV file paths within the specified directory
    csv_files = glob(os.path.join(image_path, '**', '*.csv'), recursive=True)

    # Function to process a single CSV file
    def process_file(file):
        # Extract the file name without the extension to use as the subject's name
        file_name = os.path.splitext(os.path.basename(file))[0]

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file)

        # Add a new column to the DataFrame for the subject's name
        df['subjects_name'] = file_name

        return df

    # Use ThreadPoolExecutor to process files in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_file, file) for file in csv_files]

        # As the futures complete, append the resulting DataFrames to the list
        for future in as_completed(futures):
            df_list.append(future.result())

    # Concatenate all the DataFrames in the list into a single DataFrame
    open_facs_encoded = pd.concat(df_list, ignore_index=True)

    # Set the 'subjects_name' column as the index of the DataFrame
    open_facs_encoded.set_index('subjects_name', inplace=True)

    # Save the merged DataFrame to the specified CSV file path
    open_facs_encoded.to_csv(csv_save_path)

    print("Merged DataFrame saved to:", csv_save_path)
    return open_facs_encoded

In [13]:
dpd_df = merge_openface_csvs(OPENFACE_IMG_DPD, DPD_OPENFACE_CSV)
dpd_df.columns = [column.strip() for column in dpd_df.columns]
dpd_df.head()

Merged DataFrame saved to: C:/Users/psxah20/Desktop/Dissertation/DPD_OPENFACS/DPD_OPENFACS.csv


,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
subjects_name,,,,,,,,,,,,,,,,,,,,,
DPD_1_AF10_p2c,1,0,0.0,0.98,1,0.399833,0.282901,-0.871837,-0.273370,0.258648,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
DPD_1_AF13_p6c,1,0,0.0,0.98,1,0.873417,0.183009,-0.451275,-0.632996,0.249555,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
DPD_1_AF10_p4c,1,0,0.0,0.98,1,0.990650,0.131064,0.037867,-0.350799,0.305312,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DPD_1_AF15_p5c,1,0,0.0,0.98,1,0.590857,0.126149,-0.796853,-0.378080,0.343957,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
DPD_1_AF13_p2c,1,0,0.0,0.98,1,0.967926,0.212458,0.134093,-0.742388,0.301905,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [524]:
def process_action_units(open_facs_encoded, original_facs_encoding):
    """
    Processes action units by selecting relevant columns from open_facs_encoded 
    and joining them with original_facs_encoding DataFrame. The function also handles 
    missing data by dropping rows with NaN values.
    
    Parameters:
    - open_facs_encoded (DataFrame): The DataFrame containing OpenFace-encoded data.
    - original_facs_encoding (DataFrame): The DataFrame containing original FACS encoding data.
    
    Returns:
    - action_unit (DataFrame): The processed DataFrame containing selected action units.
    """
    # Initialize an empty DataFrame to store action units
    action_unit = pd.DataFrame()

    # Set the index of action_unit to match open_facs_encoded
    action_unit.index = open_facs_encoded.index
    column_name = []

    # Iterate through the columns in open_facs_encoded
    for index_v in open_facs_encoded.columns:
        # Check if the column name indicates an action unit with a "_c" suffix
        if 'AU' in index_v and '_r' in index_v:
            au = (index_v.split('_')[0] + '_OF').strip()
            # If the corresponding column exists in original_facs_encoding, add it to action_unit
            if au in original_facs_encoding.columns:
                if index_v in ["AU06_r","AU10_r"]:
                    action_unit[index_v] = [0.0 if values <= 3 else 1.0 for values in open_facs_encoded[index_v]]
                elif index_v == "AU07_r":
                    action_unit[index_v] = [0.0 if values <= 0.2 else 1.0 for values in open_facs_encoded[index_v]]
                elif index_v in ["AU04_r", "AU09_r", "AU45_r"]:
                    action_unit[index_v] = [0.0 if values in range(1,2) else 1.0 for values in open_facs_encoded[index_v]]
                else:
                    action_unit[index_v] = [0.0 if values == 0 else 1.0 for values in open_facs_encoded[index_v]]
                column_name.append(au.strip())

    # Join the selected action units with the original FACS encoding
    action_unit = action_unit.join(original_facs_encoding[column_name])

    # Drop rows with any missing values
    action_unit.dropna(axis=0, inplace=True)

    # Return the processed DataFrame
    return action_unit

In [525]:
dpd_action_units = process_action_units(dpd_df, original_facs_dpd)
dpd_action_units.columns = [column.strip() for column in dpd_action_units.columns]
dpd_action_units.head()

,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,...,AU10_OF,AU12_OF,AU14_OF,AU15_OF,AU17_OF,AU20_OF,AU23_OF,AU25_OF,AU26_OF,AU45_OF
DPD_1_AF10_p2c,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,...,0,0,0,0,0,0,0,1,0,1
DPD_1_AF10_p4c,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,1,0
DPD_1_AF10_p6c,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,1,1
DPD_1_AF11_p6c,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,...,0,0,1,0,0,0,0,0,0,1
DPD_1_AF12_p3c,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1,1,0,0,1,0,0,1,1,1


In [526]:
au_comaprision = manual_facs_dpd.join(dpd_action_units, on=manual_facs_dpd.index)
au_comaprision.columns = [column.strip() for column in au_comaprision.columns]
au_comaprision.dropna(inplace=True, axis=0)
au_comaprision

,Expession_Highest,Pain_Expression,"AU(4,6,7,9,45)_PainIndex",AU01_m,AU02_m,AU04_m,AU05_m,AU06_m,AU07_m,AU09_m,...,AU10_OF,AU12_OF,AU14_OF,AU15_OF,AU17_OF,AU20_OF,AU23_OF,AU25_OF,AU26_OF,AU45_OF
Target,,,,,,,,,,,,,,,,,,,,,
DPD_1_AF12_p4c,surprised,2.285714,2,1,0,1,0,0,1,0,...,1,0,0,0,0,0,0,1,1,1
DPD_1_AF13_p2c,pain,3.292683,3,0,0,1,0,0,0,1,...,0,0,1,0,1,0,1,0,0,1
DPD_1_AF15_p5c,pain,4.463415,3,0,0,1,0,0,0,1,...,1,0,1,0,1,0,1,0,0,0
DPD_1_AF16_p3c,pain,3.904762,3,0,0,1,0,0,0,1,...,0,1,1,0,1,1,1,0,0,1
DPD_1_AF17_p2c,pain,4.000000,3,0,0,1,0,0,0,1,...,0,1,1,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DPD_1_WM40_p3c,surprised,2.244898,2,0,0,1,1,0,0,1,...,1,0,0,0,0,1,1,1,1,1
DPD_1_WM43_p2c,pain,4.500000,4,0,0,1,0,1,1,0,...,1,1,1,0,0,1,0,1,0,1
DPD_1_WM5_p1,pain,5.410256,5,0,0,1,0,1,1,1,...,1,0,1,0,0,1,0,1,1,1


In [538]:
results_df = pd.DataFrame()
for org in manual_facs_dpd.columns:
    if '_PainIndex' not in org:
        number = org.split('_')[0][-2:]
        if number not in  ['on','in'] and number in ["04","06","07","09","10","45"]:
            comparison_result_OF = au_comaprision[f'AU{number}_m'] == au_comaprision[f'AU{number}_OF']
            if number in ["09","45"]:
                comparison_result_c = au_comaprision[f'AU{number}_m'] == au_comaprision[f'AU{number}_OF']
            else:
                comparison_result_c = au_comaprision[f'AU{number}_m'] == au_comaprision[f'AU{number}_r']
            
            # Calculate accuracy
            accuracy_OF = comparison_result_OF.sum() / len(comparison_result_OF)
            accuracy_c = comparison_result_c.sum() / len(comparison_result_c)
            results_df.loc[f'AU{number}',"Accuracy_Previous_Study"] = accuracy_OF
            results_df.loc[f'AU{number}',"Accuracy_New_Values"] = accuracy_c

results_df.index.name = 'Action_Units'

results_df.to_csv("C:/Users/psxah20/Desktop/Dissertation/DPD_OPENFACS/OpenFACS_ACTION_UNITS.csv", header=True)
results_df.head()

,Accuracy_Previous_Study,Accuracy_New_Values
Action_Units,,
AU04,0.797872,0.851064
AU06,0.574468,0.765957
AU07,0.776596,0.840426
AU09,0.723404,0.723404
AU10,0.414894,0.723404


In [ ]:
# Randomly select 6 unique indices
random_indices = random.sample(range(len(DPD_IMAGES_PATH)), 6)

# Paths for cropped images and landmarks
cropped_images = glob(os.path.join(OPENFACE_IMG_DPD, '**', '*.bmp'), recursive=True)
landmarks_cal_faces = glob(os.path.join(OPENFACE_IMG_DPD, '**', '*.avi'), recursive=True)

# Create the plot with 6 rows and 3 columns
fig, axes = plt.subplots(6, 3, figsize=(15, 20))

for i, idx in enumerate(random_indices):
    # Row 1: Original image
    original_image = mpimg.imread(DPD_IMAGES_PATH[idx])
    axes[i, 0].imshow(original_image)
    axes[i, 0].axis('off')
    axes[i, 0].set_title(f'Original {i+1}')
    
    # Row 2: Cropped image
    cropped_image = mpimg.imread(cropped_images[idx])
    axes[i, 1].imshow(cropped_image)
    axes[i, 1].axis('off')
    axes[i, 1].set_title(f'Cropped {i+1}')
    
    # Row 3: Landmarks identified face
    # Using OpenCV to capture the first frame from the AVI file
    cap = cv2.VideoCapture(landmarks_cal_faces[idx])
    ret, frame = cap.read()
    cap.release()
    if ret:
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        axes[i, 2].imshow(frame_rgb)
    axes[i, 2].axis('off')
    axes[i, 2].set_title(f'Landmarks {i+1}')

plt.tight_layout()
# Save the plot to the specified path
output_path = "P:/Dissertation/Output plots/open_facs.png"
plt.savefig(output_path)

plt.show()